# [실습] Single Agent 구축부터 MCP(Model Context Protocol) 연동까지

이 노트북은 AI Agent의 동작 원리(ReAct)를 밑바닥부터 이해하고, smolagents를 활용하며, MCP을 통해 외부 도구를 확장하는 전체 과정을 다룹니다.

> **학습 목표:**
> 1.  **Handcrafted Agent:** `Prompt Engineering`과 `Python`만으로 에이전트의 사고 루프(Reasoning Loop)를 직접 구현합니다.
> 2.  **Smolagents:** Hugging Face의 최신 라이브러리를 통해 **Code Agent**의 강력함을 체험합니다.
> 3.  **MCP Integration:** "LLM의 USB-C"라 불리는 MCP 서버를 직접 구축하고, 에이전트와 연동하여 도구를 확장합니다.

## 0. 환경 설정 (Setup)

실습에 필요한 라이브러리를 설치합니다.
* `smolagents`: 가벼운 에이전트 프레임워크
* `mcp`, `fastmcp`: MCP 서버 및 클라이언트 구현체
* `openai`: LLM 백엔드 (또는 anthropic 등)

In [1]:
# 필요 패키지 설치 (실행 후 커널 재시작이 필요할 수 있습니다)
%pip install -q openai smolagents mcp fastmcp python-dotenv nest_asyncio

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import nest_asyncio
from dotenv import load_dotenv

# Jupyter에서 비동기 루프 실행을 위한 설정 (MCP 통신용)
nest_asyncio.apply()

# API Key 설정 (OpenAI 권장)
load_dotenv()
if not os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = "API Key를 입력하세요!"

---

## 1. Handcrafted Agent: ReAct 패턴의 해부

> **이론 배경:**
> ReAct(Reasoning + Acting)는 **Thought(사고) → Action(행동) → Observation(관찰)**의 루프를 돕니다.
> 프레임워크 없이 이를 구현해보면, 에이전트가 어떻게 `Stop Sequence`를 인식하고 도구를 호출하는지 정확히 알 수 있습니다.

### 1-1. 도구(Tools) 및 프롬프트 정의

에이전트에게 "시간 확인"과 "계산" 능력을 부여합니다.

In [4]:
import datetime
import json
from zoneinfo import ZoneInfo

# 1. 도구 정의 (실제 함수)
def get_current_time(timezone: str = "Asia/Seoul") -> str:
    """현재 시간을 반환합니다. timezone 예: 'UTC', 'Asia/Seoul'"""
    try:
        # [FIX] ZoneInfo를 사용하여 실제 timezone을 적용
        now = datetime.datetime.now(ZoneInfo(timezone))
        return now.strftime(f"%Y-%m-%d %H:%M:%S ({timezone})")
    except Exception:
        return "Invalid Timezone"

def calculator(expression: str) -> str:
    """수식을 문자열로 받아 계산합니다. 예: '2 * 3 + 1'"""
    try:
        # 보안상 eval은 위험하지만 예제 단순화를 위해 사용
        return str(eval(expression, {"__builtins__": None}, {}))
    except Exception as e:
        return f"Error: {e}"

# 도구 매핑 테이블
tools_map = {
    "get_current_time": get_current_time,
    "calculator": calculator
}

# 2. System Prompt (ReAct Style)
REACT_SYSTEM_PROMPT = """
You are a helpful AI Assistant.
To answer the user's question, you function in a loop of Thought, Action, and Observation.

AVAILABLE TOOLS:
- get_current_time(timezone): Get current time. Example args: {"timezone": "Asia/Seoul"}
- calculator(expression): Calculate math expression. Example args: {"expression": "2 * 3 + 1"}

FORMAT INSTRUCTIONS:
Use the following format strictly:

Question: the input question you must answer
Thought: you should always think about what to do
Action:
```json
{
  "tool_name": "tool_name_here",
  "arguments": {"param_name": "value"}
}
```
Observation: the result of the action (provided by system)
... (this Thought/Action/Observation can repeat N times)
Final Answer: the final answer to the original input question

IMPORTANT: The "arguments" field must always be a JSON object (dict), not a string.

Begin!
"""

### 1-2. ReAct Agent Class 구현 (The Core Loop)

LLM의 응답을 파싱하고, 도구를 실행한 뒤, 그 결과를 다시 프롬프트에 붙여서 LLM에게 던져주는 **핵심 로직**입니다.

In [6]:
import re
from openai import OpenAI

class HandcraftedAgent:
    def __init__(self, system_prompt, tools_map, model="gpt-4.1"):
        self.client = OpenAI()
        self.system_prompt = system_prompt
        self.tools_map = tools_map
        self.model = model
        self.max_steps = 5  # 무한 루프 방지

    def chat(self, user_input):
        # 대화 기록(Context) 초기화
        messages = [
            {"role": "system", "content": self.system_prompt},
            {"role": "user", "content": f"Question: {user_input}"}
        ]
        
        print(f"\U0001f680 [Start Task]: {user_input}")

        for step in range(self.max_steps):
            # 1. LLM 호출
            response = self.client.chat.completions.create(
                model=self.model,
                messages=messages,
                stop=["Observation:"] # LLM이 스스로 관찰 결과를 지어내지 못하게 멈춤
            )
            llm_output = response.choices[0].message.content
            print(f"\n--- Step {step+1} ---\n{llm_output}")

            # 메시지 기록에 LLM의 생각 추가
            messages.append({"role": "assistant", "content": llm_output})

            # 2. 종료 조건 확인 (Final Answer)
            if "Final Answer:" in llm_output:
                return llm_output.split("Final Answer:")[-1].strip()

            # 3. Action 파싱 (JSON 추출)
            # Markdown 코드 블록 ```json ... ``` 사이의 내용을 추출
            json_match = re.search(r"```json\s*(\{.*?\})\s*```", llm_output, re.DOTALL)
            
            if json_match:
                try:
                    action_data = json.loads(json_match.group(1))
                    tool_name = action_data.get("tool_name")
                    args = action_data.get("arguments")

                    # 4. 도구 실행 (Execution)
                    print(f"\u2699\ufe0f [Tool Call]: {tool_name} with args {args}")
                    if tool_name in self.tools_map:
                        # [FIX] args가 dict이면 **kwargs로 전달, 문자열이면 그대로 전달
                        if isinstance(args, dict):
                            observation = self.tools_map[tool_name](**args)
                        else:
                            observation = self.tools_map[tool_name](args)
                    else:
                        observation = f"Error: Tool '{tool_name}' not found."

                except Exception as e:
                    observation = f"Error parsing/executing action: {str(e)}"
            else:
                observation = "Error: Invalid Action Format. Please provide Action in JSON block."

            # 5. 관찰 결과(Observation)를 Context에 주입
            print(f"\U0001f440 [Observation]: {observation}")
            
            # 다음 턴을 위해 User 메시지로 관찰 결과 전달
            messages.append({"role": "user", "content": f"Observation: {observation}"})

        return "Max steps reached without final answer."

# --- 실행 테스트 ---
agent = HandcraftedAgent(REACT_SYSTEM_PROMPT, tools_map)
result = agent.chat("지금 서울 시간을 확인하고, 해당 연도에 2025를 더하면 몇인지 계산해줘.")
print(f"\n\U0001f3c1 [Final Result]: {result}")

🚀 [Start Task]: 지금 서울 시간을 확인하고, 해당 연도에 2025를 더하면 몇인지 계산해줘.

--- Step 1 ---
Thought: 먼저 서울의 현재 연도를 확인해야 한다. 그 다음 해당 연도에 2025를 더해서 결과를 계산해야 한다.
Action:
```json
{
  "tool_name": "get_current_time",
  "arguments": {"timezone": "Asia/Seoul"}
}
```

⚙️ [Tool Call]: get_current_time with args {'timezone': 'Asia/Seoul'}
👀 [Observation]: 2026-02-07 10:03:07 (Asia/Seoul)

--- Step 2 ---
Thought: 현재 서울의 연도는 2026년이다. 이제 2026에 2025를 더해서 결과를 계산해야 한다.
Action:
```json
{
  "tool_name": "calculator",
  "arguments": {"expression": "2026 + 2025"}
}
```

⚙️ [Tool Call]: calculator with args {'expression': '2026 + 2025'}
👀 [Observation]: 4051

--- Step 3 ---
Final Answer: 지금 서울의 연도는 2026년이고, 여기에 2025를 더하면 4051입니다.

🏁 [Final Result]: 지금 서울의 연도는 2026년이고, 여기에 2025를 더하면 4051입니다.


---

## 2. Using Framework: smolagents (Code Agent)

> **이론 배경:**
> `smolagents`는 LLM이 JSON이 아닌 **Python Code**를 직접 생성하여 도구를 호출하는 **Code Agent** 방식을 사용합니다.
> 이는 복잡한 논리(Loop, If문)를 표현하는 데 JSON보다 훨씬 효율적입니다.

### 2-1. Python 함수 기반 도구 생성
`@tool` 데코레이터를 사용하여 함수를 도구로 변환합니다. Docstring과 Type Hint가 매우 중요합니다.

In [7]:
from smolagents import CodeAgent, Tool, LiteLLMModel, tool

# 기존 함수를 smolagents용 도구로 래핑
@tool
def check_server_status(server_name: str) -> str:
    """
    Checks the health status of a specific server.
    
    Args:
        server_name: The name of the server to check (e.g., 'prod-db', 'web-01').
    """
    # 더미 로직
    if "prod" in server_name:
        return f"[ALERT] {server_name} is experiencing high load (CPU 95%)."
    return f"{server_name} is running smoothly."

@tool
def restart_server(server_name: str) -> str:
    """
    Restarts the specified server. Use this only when the server status is bad.
    
    Args:
        server_name: The name of the server to restart.
    """
    return f"Success: {server_name} has been restarted and is now healthy."

### 2-2. Code Agent 실행
`CodeAgent`는 내부적으로 **Python Interpreter**를 가지고 있어, LLM이 짠 코드를 샌드박스 환경에서 안전하게 실행합니다.

In [8]:
# 모델 설정 (OpenAI gpt-4o 사용)
model = LiteLLMModel(model_id="gpt-4.1")

# 에이전트 생성
manager_agent = CodeAgent(
    tools=[check_server_status, restart_server], # 우리가 만든 도구들
    model=model,
    add_base_tools=True, # 검색(DuckDuckGo) 등 기본 도구 포함
    max_steps=6
)

print("--- smolagents: Server Manager Scenario ---")
# 시나리오: 상태 확인 후 문제가 있으면 재시작까지 수행하는 복합 작업
response = manager_agent.run(
    "Check the status of 'prod-db'. If there is a problem, fix it and confirm the status again."
)

print(f"\n\U0001f916 Agent Answer: {response}")

--- smolagents: Server Manager Scenario ---


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Check the status of 'prod-db'. If there is a problem, fix it and confirm the status again.                      │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4.1 ────────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  status = check_server_status('prod-db')                                                                          
  print(status)                                                                                                    
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
[ALERT] prod-db is experiencing high load (CPU 95%).

Out: None

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("After restarting, 'prod-db' is still experiencing high load (CPU 95%). The problem persists and    
  may require more in-depth investigation or intervention beyond a restart.")                                      
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: After restarting, 'prod-db' is still experiencing high load (CPU 95%). The problem persists and
may require more in-depth investigation or intervention beyond a restart.

[Step 3: Duration 3.55 seconds| Input tokens: 6,859 | Output tokens: 268]


🤖 Agent Answer: After restarting, 'prod-db' is still experiencing high load (CPU 95%). The problem persists and may require more in-depth investigation or intervention beyond a restart.


---

## 3. Tool Access: MCP (Model Context Protocol) 연동

> **이론 배경:**
> **MCP**는 LLM과 도구 사이의 표준 프로토콜입니다.
> - **기존:** 모든 앱(LangChain, Semantic Kernel 등)마다 도구 연결 코드를 따로 짜야 함.
> - **MCP:** 서버 하나만 만들어두면 Claude Desktop, Cursor, 그리고 우리가 만든 Agent 어디서든 연결 가능 (Plug-and-Play).

### 3-1. MCP Server 구축 (file: `myserver.py`)
`fastmcp`를 사용하여 별도의 프로세스로 동작하는 MCP 서버를 만듭니다. 주피터 환경 한계를 극복하기 위해 `%%writefile` 매직 커맨드로 파일을 생성합니다.

In [9]:
%%writefile myserver.py
import json
from fastmcp import FastMCP

# 1. MCP 서버 인스턴스 생성
mcp = FastMCP("Advanced Data Tools")

# 2. 도구 등록
@mcp.tool()
def analyze_log_file(filepath: str) -> str:
    """
    Analyzes a log file and returns a summary of errors.
    Mock function for demo.
    """
    return f"Checked {filepath}: Found 3 Critical Errors, 12 Warnings."

@mcp.tool()
def query_customer_db(customer_id: str) -> str:
    """
    Retrieves customer information from the secure database.
    Returns a JSON string of the customer record.
    """
    # [FIX] dict 대신 JSON 문자열을 반환하여 MCP text 응답과 호환
    if customer_id == "C123":
        return json.dumps({"name": "Kim Agent", "plan": "Premium", "last_login": "2024-02-07"})
    return json.dumps({"error": "Customer not found"})

# 3. 서버 실행
if __name__ == "__main__":
    mcp.run()

Writing myserver.py


### 3-2. MCP Client Adapter 구현 (Crucial Step)
`smolagents`는 기본적으로 MCP를 직접 지원하지 않으므로, **MCP Client → Python Function**으로 변환해주는 어댑터(Adapter)를 구현해야 합니다. 

In [12]:
import asyncio
from smolagents import Tool

class MCPAdapterSSE:
    """HTTP/SSE 방식으로 MCP 서버와 통신하는 어댑터 (Windows Jupyter 호환)"""
    
    def __init__(self, base_url="http://localhost:8000"):
        self.base_url = base_url.rstrip("/")
    
    def get_smolagents_tools(self):
        loop = asyncio.get_event_loop()
        return loop.run_until_complete(self._fetch_tools())
    
    async def _fetch_tools(self):
        from mcp.client.sse import sse_client
        from mcp import ClientSession
        
        async with sse_client(self.base_url + "/sse") as (read, write):
            async with ClientSession(read, write) as session:
                await session.initialize()
                result = await session.list_tools()
                
                tools = []
                for mcp_tool in result.tools:
                    tools.append(self._create_wrapper(mcp_tool))
                return tools
    
    def _create_wrapper(self, mcp_tool):
        tool_name = mcp_tool.name
        tool_desc = mcp_tool.description or "No description."
        base_url = self.base_url
        
        def call_tool(**kwargs):
            async def _call():
                from mcp.client.sse import sse_client
                from mcp import ClientSession
                
                async with sse_client(base_url + "/sse") as (read, write):
                    async with ClientSession(read, write) as session:
                        await session.initialize()
                        result = await session.call_tool(tool_name, arguments=kwargs)
                        if result.content:
                            return result.content[0].text
                        return "No content returned."
            
            loop = asyncio.get_event_loop()
            return loop.run_until_complete(_call())
        
        return Tool.from_function(call_tool, name=tool_name, description=tool_desc)

### 3-3. Final Integration: Agent + MCP

이제 직접 만든 `myserver.py`를 Agent에 연결합니다. 마치 USB 장치를 꽂듯이 도구가 확장되는 것을 확인하세요.

In [ ]:
# ⚠️ 먼저 별도 터미널에서 MCP 서버를 실행하세요:
#   fastmcp run myserver.py --transport sse --port 8000

# 1. SSE 어댑터로 연결
print("🔌 Connecting to MCP Server (SSE)...")
adapter = MCPAdapterSSE("http://localhost:8000")

# 2. 도구 로드
mcp_tools = adapter.get_smolagents_tools()
print(f"✅ Loaded Tools from MCP: {[t.name for t in mcp_tools]}")

# 3. Agent 생성
final_agent = CodeAgent(
    tools=mcp_tools,
    model=model,
    add_base_tools=True
)

print("\n🚀 [Scenario]: Customer Support with MCP Tools")
prompt = """
Customer ID 'C123' has reported an issue.
1. Retrieve the customer's profile using the database tool.
2. Analyze the system log file named 'system_error.log'.
3. Provide a brief report combining the customer info and log analysis.
"""

result = final_agent.run(prompt)
print(f"\n📝 Final Report:\n{result}")

## 4. Summary & Next Steps

오늘 실습을 통해 우리는 핵심 개념들을 모두 코드로 구현했습니다.

1.  **Handcrafted Agent:** `Stop Sequence`와 `JSON Parsing`을 이용해 **ReAct Loop**를 직접 제어했습니다.
2.  **Smolagents:** Python 코드를 생성하여 논리적인 작업 수행이 가능한 **Code Agent**를 경험했습니다.
3.  **MCP:** 로컬 서버(`myserver.py`)를 띄우고, 이를 표준 프로토콜로 연결하여 **확장성 있는 아키텍처**를 구현했습니다.